## Sampling and optimizing binding sites on a set of water clusters


This tuturial will explain to you the prodecure to compute a BE energy distribuiton on a amorphized set 
of water clusters. In the previouse tutorial we learned how to create a OptimizationDatase. For this tutorial we asume that you already learned how to set up a [QCFractal server](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/setup_server.html) and spin up a [QCFractal manager](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/managers.html) on your server. The same insctructions are valid if you want to contribuite to the BEEP database, the only thing that changes is the URL of the server. 

So at this point you have already created a collection with a set of amporphised water clusters. We will use the set consisting of 12 water molecules constructed in the previous tutorial. You also have an OptimizationDataset which contains the optimized geometry of the samll molecule you want to sample, for this example we will use methanol.

The total procedure consists of three steps, **sampling the water clusters**, **computing the BSSE corrected binding energy for each unique binding site** and **computing the hessian** for the binding site of one of the sampled clusters to include the zero-point vibrational energy in the binding energy calculation. For each one of those steps there is a dedicated CLI. 

The first step is to sample the surface of the water clusters that span the model surface. With this purpose, `launch_sampling` creates two OptimiationDataSets:  One that contains initial sampled structures (with the prefix *pre*) and a second one where the unique optimized structures are stored after the initial optimization at 
the sampling level of theory. After the initial optimization is done, the geometry of the binding sites is refined using a HF-3C/MINIX level of theory. Nontheless the geometries can also be further optimized (e.g. at a DFT hybrid or meta-hyrbid GGA level of theory) in the future as the are contained in an OptimizationDataSet. 
For the sampling procedure we developed a option-specific command line interface `launch_sampling.py`. It can be found in the script folder of this module. 

```
Usage: python launch_sampling.py [options]

A command line interface to sample the surface of a set of water clusters (stored in a 
QCFractal DataSet)  with a small molecule or atom. This CLI is part
of the Binding Energy Evaluation Platform (BEEP).


Options:
  -h, --help            show this help message and exit
  --client_address=CLIENT_ADDRESS
                        The URL address and port of the QCFractal server
                        (default: localhost:7777)
  --molecule=MOLECULE   The name of the molecule to be sampled (from a
                        QCFractal OptimizationDataSet collection)
  --surface_model_collection=SURFACE_MODEL_COLLECTION
                        The name of the collection with the set of water
                        clusters (dafault: ASW_22)
  --small_molecule_collection=SMALL_MOLECULE_COLLECTION
                        The name of the collection containing molecules or
                        radicals (dafault: Small_molecules)
  --molecules_per_round=MOLECULE_PER_ROUND
                        Number of molecules to be optimized each round
                        (Default = 10)
  --sampling_shell=SAMPLING_SHELL
                        The shell size of sampling space (Default = 1.5
                        Angstrom)
  --maximal_binding_sites=MAXIMAL_BINDING_SITES
                        The maximal number of binding sites per cluster
                        (default: 21)
  -l LEVEL_OF_THEORY, --level_of_theory=LEVEL_OF_THEORY
                        The level of theory in the format: method_basis
                        (default: blyp_def2-svp)
  --refinement_level_of_theory=R_LEVEL_OF_THEORY
                        The level of theory for geometry refinement in the
                        format: method_basis (default: hf3c_minix)
  --rmsd_value=RMSD_VAL
                        Rmsd geometrical criteria, all structure below this
                        value will not be considered as unique. (default: 0.40
                        angstrom)
  --rmsd_symmetry       Consider the molecular symmetry for the rmsd
                        calculation
  -p PROGRAM, --program=PROGRAM
                        The program to use for this calculation (default:
                        psi4)
  -k KEYWORD_ID, --keyword_id=KEYWORD_ID
                        ID of the QC keywords for the OptimizationDataSet
                        specification (default: None)
  --print_out           Print an output


```

The `SURFACE_MODEL_COLLECTION` refers to the collection 
that contains the set of clusters that span the surface model, in our example there are three water clusters 
of 12 molecules each in the DataSet collection. 

The `SMALL_MOLECULE_COLLECTION` options refers to the collection that contains all the small molecules. 

The `SAMPLING_SHELL` referes to the width of the sampling shell around the cluster. If this number is too large 
it is possible that the molecule will take to long to converge to a binding site during the optimization

**IMPORTANT**: Before running `launch_sampling` you need to spin up two qcfractal-managers, one with a `tag=sampling` and a second with a `tag=refinement` . This module contains two examples of qcfractal-managers running using dask and slurm. For more examples and options check out  [QCFractal manager documentation](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/managers.html)

Finally you can check out the progress of the sampling prodecure by querying the OptimizationDataset asocciated 
to the sampling prodecure. 

In [ ]:
import qcfractal.interface as ptl

client = ptl.FractalClient(address="localhost:7777", verify=False)
client.list_collections()

In [ ]:
ds_opt = client.get_collection("OptimizationDataset", "ch3oh_w12_01")
ds_opt.status()